In [2]:
import pandas as pd
import numpy as np
import os

import dgl
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0, 1"
import torch
import numpy as np
from gensim.models import Word2Vec
from tqdm import tqdm

BASE_PATH = 'Z:/DataScience/DGL'
import pickle
with open(os.path.join(BASE_PATH, 'labels.pkl'), 'rb') as f:
    label_data = pickle.load(f)

labels = torch.from_numpy(label_data['label'])

graphs, _ = dgl.load_graphs(os.path.join(BASE_PATH, 'graph.bin'))
graph = graphs[0]
degrees = graph.out_degrees() + graph.in_degrees()
graph = dgl.to_bidirected(graph, copy_ndata=True)
graph = dgl.add_self_loop(graph)
print('################ Graph info: ###############')

print(np.mean(graph.in_degrees().numpy()))
print(np.mean(graph.out_degrees().numpy()))

EMB_SIZE = 32

nodes = []
for repeat in range(10):
    nodes.extend([i for i in range(graph.num_nodes())])



################ Graph info: ###############
16.915180393560085
16.915180393560085


In [3]:
node_feat = np.load(BASE_PATH + '/features.npy')
node_info = np.load(BASE_PATH + '/node_info.npy')

In [5]:
# from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [6]:
%%time
sample_times = 50
nodes = []
num_nodes  = int(graph.num_nodes())
useful_index = torch.where(labels[:num_nodes] != -1)[0].numpy()
train_index, valid_index = train_test_split(useful_index, test_size=0.2)
print(num_nodes)
walk_label_features = []
preds = []
for i in range(num_nodes):
    for j in range(sample_times):
        nodes.append(i)
s = dgl.sampling.node2vec_random_walk(graph, nodes, 0.1, 1, walk_length=10)

3655452
Wall time: 10min 24s


In [7]:
for k in tqdm(range(num_nodes)):
    select_index = s[k *sample_times : (k+1)*sample_times, 1:] != s[sample_times*k][0]
    select_nei = s[k *sample_times : (k+1)*sample_times, 1:][select_index]
    hop5_nei_labels = labels[select_nei]
    hop5_labels = torch.zeros(hop5_nei_labels.shape[0], 24)
    if len(hop5_nei_labels) == 0:
        walk_label_features.append(torch.zeros(1, 24))
    else:
        hop5_labels[np.arange(hop5_labels.shape[0]).reshape(-1, 1), hop5_nei_labels.numpy().reshape(-1, 1)] = 1
        mean_hop5_labels = hop5_labels.mean(0)
        walk_label_features.append(mean_hop5_labels.unsqueeze(0))
        if labels[s[sample_times*k][0]] == -1:
            continue
        preds.append(labels[s[sample_times*k][0]] in mean_hop5_labels[:-1].topk(1).indices)
walk_label_features = torch.cat(walk_label_features)
# np.save(BASE_PATH + f'/walklength{10}_sampletimes{sample_times}.npy', walk_label_features)
print(np.mean(preds))

100%|██████████████████████████████████████████████████████████████████████| 3655452/3655452 [10:10<00:00, 5992.11it/s]


0.5857910480951597


In [12]:
walk_label_features = walk_label_features.cpu().numpy()
np.save('walk_label_features.npy',walk_label_features)

In [13]:
walk_label_features = np.load('walk_label_features.npy')
print(walk_label_features)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.00408163 0.01632653 ... 0.0122449  0.         0.6530612 ]
 [0.0020284  0.09330629 0.0020284  ... 0.0040568  0.         0.78498983]
 ...
 [0.         0.         0.01830664 ... 0.         0.         0.49427918]
 [0.         0.         0.         ... 0.         0.24130435 0.6956522 ]
 [0.         0.         0.         ... 0.         0.         0.95110023]]


In [37]:
innode_feat_mean = node_info[:,:25]/node_info[:,:25].sum(axis=-1).reshape(-1, 1)
outnode_feat_mean = node_info[:,25:]/node_info[:,25:].sum(axis=-1).reshape(-1, 1)
node_feat_mean = innode_feat_mean + outnode_feat_mean

/data/anaconda3/envs/torch1.8.1/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.
/data/anaconda3/envs/torch1.8.1/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [40]:
train_x = walk_label_features[train_index].numpy()
train_ori_features = node_feat[train_index]
train_node_info = node_info[train_index]
# train_innode_feat_mean = innode_feat_mean[train_index]
# train_outnode_feat_mean = outnode_feat_mean[train_index]
# train_node_feat_mean = node_feat_mean[train_index]
train_x = np.concatenate([train_x, train_ori_features, train_node_info], axis=-1)

train_y = labels[train_index].numpy()

valid_x = walk_label_features[valid_index].numpy()
valid_ori_features = node_feat[valid_index]
valid_node_info = node_info[valid_index]

# valid_innode_feat_mean = innode_feat_mean[valid_index]
# valid_outnode_feat_mean = outnode_feat_mean[valid_index]
# valid_node_feat_mean = node_feat_mean[valid_index]

valid_x = np.concatenate([valid_x, valid_ori_features, valid_node_info], axis=-1)

valid_y = labels[valid_index].numpy()

In [41]:
cbt = CatBoostClassifier(iterations=4000,learning_rate=0.2,task_type='GPU', loss_function='MultiClass', eval_metric='Accuracy')

In [42]:
cbt.fit(
    train_x, train_y,
    eval_set=(valid_x, valid_y),
    verbose=100,
)

0:	learn: 0.3594257	test: 0.3602478	best: 0.3602478 (0)	total: 284ms	remaining: 18m 54s
100:	learn: 0.5696519	test: 0.5675495	best: 0.5675495 (100)	total: 14.5s	remaining: 9m 18s
200:	learn: 0.5910455	test: 0.5856073	best: 0.5856073 (200)	total: 30.5s	remaining: 9m 37s
300:	learn: 0.6014293	test: 0.5927261	best: 0.5927261 (300)	total: 46.9s	remaining: 9m 36s
400:	learn: 0.6088616	test: 0.5963741	best: 0.5964124 (395)	total: 1m 3s	remaining: 9m 28s
500:	learn: 0.6146819	test: 0.5981980	best: 0.5983177 (495)	total: 1m 19s	remaining: 9m 15s
600:	learn: 0.6191736	test: 0.5995912	best: 0.5996199 (599)	total: 1m 35s	remaining: 9m
700:	learn: 0.6236606	test: 0.6011040	best: 0.6011901 (696)	total: 1m 51s	remaining: 8m 44s
800:	learn: 0.6279967	test: 0.6020854	best: 0.6021045 (796)	total: 2m 7s	remaining: 8m 28s
900:	learn: 0.6321163	test: 0.6030716	best: 0.6031529 (891)	total: 2m 23s	remaining: 8m 12s
1000:	learn: 0.6359486	test: 0.6031960	best: 0.6033971 (973)	total: 2m 39s	remaining: 7m 57s


KeyboardInterrupt: 

## n2V

In [7]:
print("sampling complete")
walks = walks.numpy().tolist()
print("training")
model = Word2Vec(walks, vector_size=EMB_SIZE, window=10, min_count=0, sg=1, workers=12)
print("training complete")
w2v = np.zeros([len(model.wv.index_to_key), EMB_SIZE])
print(w2v.shape)
for i, index_id in enumerate(sorted(model.wv.index_to_key)):
    w2v[i] = model.wv[int(index_id)]

with open(os.path.join(BASE_PATH, 'features_n2v.npy'), 'wb') as f:
    np.save(f, w2v)

print(w2v.shape)


features = np.load(os.path.join(BASE_PATH, 'features.npy'))
print(features.shape)
features = np.hstack([features,w2v])
with open(os.path.join(BASE_PATH, 'features_with_n2v.npy'), 'wb') as f:
    np.save(f, features)

sampling complete
training
training complete
(3655452, 32)
(3655452, 32)
(3655452, 300)
